In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import jline.solvers.mva.*
import jline.util.matrix.*

In [ ]:
// Nested Fork-Join Network
val model = Network("model")

In [ ]:
// Create nested fork-join structure
val delay = Delay(model, "Delay1")
val queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)
val queue2 = Queue(model, "Queue2", SchedStrategy.FCFS)
val fork = Fork(model, "Fork")
val join = Join(model, "Join", fork)
val queue3 = Queue(model, "Queue3", SchedStrategy.FCFS)
val queue4 = Queue(model, "Queue4", SchedStrategy.FCFS)
val fork2 = Fork(model, "Fork2")
val join2 = Join(model, "Join2", fork2)

val jobclass1 = ClosedClass(model, "class1", 1, delay, 0)

In [ ]:
// Service configurations
queue1.setService(jobclass1, Exp(1.0))
queue2.setService(jobclass1, Exp(1.0))
delay.setService(jobclass1, Exp(0.5))
queue3.setService(jobclass1, Exp(2.0))
queue4.setService(jobclass1, Exp(2.0))

In [ ]:
// Nested fork-join routing
val P = model.initRoutingMatrix()

P.set(jobclass1, jobclass1, delay, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
// Nested structure: queue1 goes to fork2
P.set(jobclass1, jobclass1, queue1, fork2, 1.0)
P.set(jobclass1, jobclass1, fork2, queue3, 1.0)
P.set(jobclass1, jobclass1, fork2, queue4, 1.0)
P.set(jobclass1, jobclass1, queue3, join2, 1.0)
P.set(jobclass1, jobclass1, queue4, join2, 1.0)
P.set(jobclass1, jobclass1, join2, join, 1.0)
// Queue2 goes directly to join
P.set(jobclass1, jobclass1, queue2, join, 1.0)
P.set(jobclass1, jobclass1, join, delay, 1.0)

model.link(P)

In [ ]:
// Solve with multiple methods
val solvers = mutableListOf<NetworkSolver>()

solvers.add(JMT(model, "seed", 23000))
// Add MVA with special fork-join options
solvers.add(MVA(model, "fork_join", "fjt"))

for ((i, solver) in solvers.withIndex()) {
    println("SOLVER ${i+1}: ${solver.name}")
    try {
        val avgTable = solver.avgTable
        avgTable.print()
    } catch (e: Exception) {
        println("Error: ${e.message}")
    }
}

This nested fork-join model demonstrates:

1. **Hierarchical Fork-Join Structure**:
   - **Outer Fork-Join**: Fork → {Queue1, Queue2} → Join
   - **Inner Fork-Join**: Fork2 → {Queue3, Queue4} → Join2
   - Queue1 contains the nested structure, Queue2 is simple

2. **Asymmetric Branch Complexity**:
   - **Branch 1**: Queue1 → Fork2 → {Queue3, Queue4} → Join2 → Join
   - **Branch 2**: Queue2 → Join (direct path)
   - Different levels of parallelism in each branch

3. **Service Rate Hierarchy**:
   - **Delay**: Rate 0.5 (bottleneck)
   - **Queue1, Queue2**: Rate 1.0 (moderate)
   - **Queue3, Queue4**: Rate 2.0 (fast inner processing)

4. **Synchronization Dependencies**:
   - **Inner sync**: Join2 waits for Queue3 and Queue4
   - **Outer sync**: Join waits for Join2 and Queue2
   - Creates cascading synchronization effects

5. **Performance Characteristics**:
   - **Complex branch dominance**: Branch 1 has more processing stages
   - **Multiple bottlenecks**: Both inner and outer synchronization points
   - **Amplified delays**: Nested structure compounds waiting times

6. **FCFS Scheduling**:
   - All queues use First-Come-First-Served
   - Can create blocking effects in nested structure
   - Order preservation through complex routing

The model represents:
- **Hierarchical manufacturing**: Assembly with sub-assembly processes
- **Software systems**: Nested parallel processing with sub-tasks
- **Distributed computing**: Map-Reduce with nested operations
- **Project management**: Tasks with parallel sub-projects

Key insights:
- **Nesting amplifies delays**: Each level of nesting adds synchronization overhead
- **Asymmetric performance**: Simple branch (Queue2) may wait for complex branch
- **Solver complexity**: Advanced fork-join methods (fjt) may be needed for accurate analysis

**Analysis Note**: This model uses the Fork-Join Theorem (fjt) method in MVA for better handling of nested structures, as standard methods may not capture the full complexity of hierarchical synchronization.